<b>Instalando e Carregando Pacotes</b>

In [1]:
#Instala o Pacote - Download dos datasets Imdb c/sqlite
!pip install -q imdb-sqlite

In [2]:
# Nome dos Paises formato ISO
!pip install -q pycountry

In [3]:
#importando os pacotes
# re - expressões regulares
import re
#time - tempo de execução, datas
import time
#sqlite3 - manipulação banco de dados
import sqlite3
#pycountry - Nome dos Paises formato ISO
import pycountry
#numpy - manipulação de dados
import numpy as np
#pandas - manipulação de dados (tabelas)
import pandas as pd
#matplotlib.pyplot  - graficos
import matplotlib.pyplot as plt
#seaborn - graficos
import seaborn as sns
#matplotlib (cm) - graficos
from matplotlib import cm
#sklearn ( countvectorizer) - machine learning - vetor para criar calculos
from sklearn.feature_extraction.text import CountVectorizer
#warning - alertas
import warnings
#ignorando os warnings
warnings.filterwarnings("ignore")
#definindo padrao do grafico
sns.set_theme(style = "whitegrid")

<b>CARREGANDO OS DADOS</b>

*baixando os dados*

In [4]:
#baixando os datasets
#%%time
!imdb-sqlite

2021-09-09 11:48:12,087 GET https://datasets.imdbws.com/name.basics.tsv.gz -> downloads\name.basics.tsv.gz
2021-09-09 11:48:21,041 GET https://datasets.imdbws.com/title.basics.tsv.gz -> downloads\title.basics.tsv.gz
2021-09-09 11:48:28,995 GET https://datasets.imdbws.com/title.akas.tsv.gz -> downloads\title.akas.tsv.gz
2021-09-09 11:48:38,510 GET https://datasets.imdbws.com/title.principals.tsv.gz -> downloads\title.principals.tsv.gz
2021-09-09 11:48:51,445 GET https://datasets.imdbws.com/title.episode.tsv.gz -> downloads\title.episode.tsv.gz
2021-09-09 11:48:52,574 GET https://datasets.imdbws.com/title.ratings.tsv.gz -> downloads\title.ratings.tsv.gz
2021-09-09 11:48:52,970 Populating database: imdb.db
2021-09-09 11:48:52,977 Applying schema
2021-09-09 11:48:52,995 Importing file: downloads\name.basics.tsv.gz
2021-09-09 11:48:52,995 Reading number of rows ...
2021-09-09 11:48:57,986 Inserting rows into table: people

 26%|##6       | 2936067/11210744 [00:29<01:18, 105617.28 rows/s]


100%|##########| 11210744/11210744 [01:40<00:00, 111764.93 rows/s]
2021-09-09 11:50:38,316 Importing file: downloads\title.basics.tsv.gz
2021-09-09 11:50:38,316 Reading number of rows ...
2021-09-09 11:50:41,656 Inserting rows into table: titles

100%|##########| 8249197/8249197 [01:26<00:00, 95503.39 rows/s]
2021-09-09 11:52:08,037 Importing file: downloads\title.akas.tsv.gz
2021-09-09 11:52:08,037 Reading number of rows ...
2021-09-09 11:52:14,137 Inserting rows into table: akas

  7%|6         | 2009501/29037681 [00:16<03:44, 120537.72 rows/s]


 23%|##2       | 6566148/29037681 [00:52<02:59, 125003.82 rows/s]


100%|##########| 29037681/29037681 [03:53<00:00, 124133.85 rows/s]
2021-09-09 11:56:08,064 Importing file: downloads\title.principals.tsv.gz
2021-09-09 11:56:08,064 Reading number of rows ...
2021-09-09 11:56:17,508 Inserting rows into table: crew

  1%|1         | 667937/46693621 [00:04<05:12, 147478.59 rows/s]


 10%|#         | 4867726/46693621 [00:32<04:31, 154322.64 rows/s]


 76%|#######6  | 35558720/46693621 [04:00<01:15, 147990.67 rows/s]


 82%|########1 | 38186172/46693621 [04:17<00:56, 149678.02 rows/s]


 88%|########7 | 40927002/46693621 [04:36<00:37, 151881.59 rows/s]


100%|##########| 46693621/46693621 [05:14<00:00, 148511.65 rows/s]
2021-09-09 12:01:31,923 Importing file: downloads\title.episode.tsv.gz
2021-09-09 12:01:31,924 Reading number of rows ...
2021-09-09 12:01:32,673 Inserting rows into table: episodes

100%|##########| 6052766/6052766 [00:33<00:00, 181756.05 rows/s]
2021-09-09 12:02:05,980 Importing file: downloads\title.ratings.tsv.gz
2021-09-09 12:02:05,980 Reading number of rows ...
2021-09-09 12:02:06,102 Inserting rows into table: ratings

100%|##########| 1185150/1185150 [00:07<00:00, 155406.19 rows/s]
2021-09-09 12:02:13,733 Creating table indices ...
2021-09-09 12:06:10,753 Import successful


In [11]:
# Conecta no banco de dados
conn = sqlite3.connect("imdb.db")

In [12]:
#Extrai a lista de tabelas
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [13]:
tabelas.head()

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes


In [14]:
#Convertendo o dataframe em uma lista
tabelas = tabelas["Table_Name"].values.tolist()

In [16]:
#Percorrendo a lista de tabelas no banco de dados e extrair o esquema de cada um
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela:", tabela)
    display(resultado)
    print("-" * 100)
    print("\n")

Esquema da tabela: people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: akas


...

----------------------------------------------------------------------------------------------------


Esquema da tabela: episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela: ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


